In [2]:
%load_ext autoreload
%autoreload 2
from palmari.processing import *
from palmari import Experiment, TifPipelineWidget
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

# Your first Experiment

In [3]:
exp = Experiment(data_folder="/Users/hverdier/palm_tools_data/DATA_FOLDER",
    export_folder="/Users/hverdier/palm_tools_data/EXPORT_FOLDER")

In [4]:
# Indicate conditions in the index_df table of the Experiment
# This can be done automatically if you create a subclass of Experiment (see below)
exp.index_df["condition"] = "A"
exp.index_df.loc[exp.index_df.file.str.contains("B"),"condition"] = "B"

In [5]:
exp.index_df

,file,ID,condition
0,CONDITION A/Experience1/CellZone1/ROI.tif,4c673d7a3babf12f,A
1,CONDITION A/Experience1/CellZone0/ROI.tif,0a4f0c4d5a770f14,A
2,CONDITION A/Experience0/CellZone2/ROI.tif,1aeb07e6d906fa6d,A
3,CONDITION A/Experience0/CellZone1/ROI.tif,272126b5e72b50a4,A
4,CONDITION A/Experience0/CellZone0/ROI#0.tif,e050273c2b28d429,A
5,CONDITION B/Exp0/cellZone2/ROI#0.tif,448d0bb5d719692d,B
6,CONDITION B/Exp0/cellZone1/ROI.tif,22ec7db7a6a63186,B
7,CONDITION B/Exp0/cellZone0/ROI.tif,2cf987b2e99bb1ed,B
8,CONDITION B/Exp1/cellZone2/ROI.tif,0628d1f1d8516a68,B
9,CONDITION B/Exp1/cellZone1/ROI.tif,662132ba6f8b14f0,B


# Make your Experiment subclass

In [6]:
class MyExp(Experiment):

    @property
    def custom_fields(self):
        return {"cell_zone":-2,"exp":-3,"condition":-4}

In [7]:
exp = MyExp(data_folder="/Users/hverdier/palm_tools_data/DATA_FOLDER",
    export_folder="/Users/hverdier/palm_tools_data/EXPORT_FOLDER")

In [8]:
exp.index_df

,file,ID,cell_zone,exp,condition
0,CONDITION A/Experience1/CellZone1/ROI.tif,4c673d7a3babf12f,CellZone1,Experience1,CONDITION A
1,CONDITION A/Experience1/CellZone0/ROI.tif,0a4f0c4d5a770f14,CellZone0,Experience1,CONDITION A
2,CONDITION A/Experience0/CellZone2/ROI.tif,1aeb07e6d906fa6d,CellZone2,Experience0,CONDITION A
3,CONDITION A/Experience0/CellZone1/ROI.tif,272126b5e72b50a4,CellZone1,Experience0,CONDITION A
4,CONDITION A/Experience0/CellZone0/ROI#0.tif,e050273c2b28d429,CellZone0,Experience0,CONDITION A
5,CONDITION B/Exp0/cellZone2/ROI#0.tif,448d0bb5d719692d,cellZone2,Exp0,CONDITION B
6,CONDITION B/Exp0/cellZone1/ROI.tif,22ec7db7a6a63186,cellZone1,Exp0,CONDITION B
7,CONDITION B/Exp0/cellZone0/ROI.tif,2cf987b2e99bb1ed,cellZone0,Exp0,CONDITION B
8,CONDITION B/Exp1/cellZone2/ROI.tif,0628d1f1d8516a68,cellZone2,Exp1,CONDITION B
9,CONDITION B/Exp1/cellZone1/ROI.tif,662132ba6f8b14f0,cellZone1,Exp1,CONDITION B


# Create pipelines

## Default

In [10]:
tp = TifPipeline.from_dict({"name":"default_pipeline"})
print(tp)

TIF Processing pipeline
-----------------------
Movie preprocessing steps :
-----------------------
Localizer :
	 Default Localizer (DefaultLocalizer) :
		 threshold_factor : 1.0

-----------------------
Localization processing steps :
-----------------------
Tracker :
	 Default tracker (Trackpy) (TrackpyTracker) :
		 max_diffusivity : 5.0




## Add some filtering

In [11]:
tp = TifPipeline.from_dict({
    "name":"test",
    "movie_preprocessors":[{"WindowPercentileFilter":{"percentile":10,"window_size":300}}]
})
print(tp)

TIF Processing pipeline
-----------------------
Movie preprocessing steps :
1/1 	 Local percentile filtering (WindowPercentileFilter) :
		 percentile : 10
		 window_size : 300

-----------------------
Localizer :
	 Default Localizer (DefaultLocalizer) :
		 threshold_factor : 1.0

-----------------------
Localization processing steps :
-----------------------
Tracker :
	 Default tracker (Trackpy) (TrackpyTracker) :
		 max_diffusivity : 5.0




## Or a stricter localizer

In [12]:
tp2 = TifPipeline.from_dict({
    "name":"stricter_than_default",
    "localizer":{"DefaultLocalizer":{"threshold_factor":1.5}},
})
print(tp2)

TIF Processing pipeline
-----------------------
Movie preprocessing steps :
-----------------------
Localizer :
	 Default Localizer (DefaultLocalizer) :
		 threshold_factor : 1.5

-----------------------
Localization processing steps :
-----------------------
Tracker :
	 Default tracker (Trackpy) (TrackpyTracker) :
		 max_diffusivity : 5.0




## Export and load pipelines

In [13]:
tp2.to_yaml("/Users/hverdier/palm_tools_data/pipeline2.yaml")
tp2 = TifPipeline.from_yaml("/Users/hverdier/palm_tools_data/pipeline2.yaml")

# Run the pipeline on an Acquisition

In [37]:
# Access files of your experiment as if the Experiment instance was a list
exp[1]

'CONDITION A/Experience1/CellZone0/ROI.tif'

In [33]:
# To start, we'll process just one file. A movie is represented by an Acquisition
acq = Acquisition(exp[1],experiment=exp, tif_pipeline=tp)

In [34]:
tp.process(acq)

Frame 19999: 4 trajectories present.


In [35]:
# That's it, you've got your localizations !
acq.locs.head()

,frame,x,y,ratio,sigma,total_intensity,t,n
0,0,6.011957,8.125726,1.027050,4.331901,3463.0,0.00,0
1,1,4.122741,5.696350,2.075699,4.171183,4822.0,0.03,1
2,2,7.116422,3.556992,1.109842,4.255278,4318.0,0.06,2
3,3,4.841610,3.477802,1.104055,4.325023,4581.0,0.09,3
4,3,6.604752,3.934262,1.048544,4.016388,5269.0,0.09,4


In [36]:
# They're stored here
acq.locs_path

'/Users/hverdier/palm_tools_data/EXPORT_FOLDER/test/CONDITION A/Experience1/CellZone0/ROI.locs'

In [38]:
# See localizations and tracks overlayed on the movie in Napari
acq.view(min_traj_length=5)

/Users/hverdier/opt/miniconda3/envs/gratin/lib/python3.8/site-packages/napari/_vispy/layers/image.py:228: UserWarning: data shape (20000, 100, 100) exceeds GL_MAX_TEXTURE_SIZE 2048 in at least one axis and will be downsampled. Rendering is currently in 3D mode.
  warnings.warn(


[########################################] | 100% Completed | 22.4s
Scaling positions when adding layer
Pixel size = 0.097


In [30]:
# Visualize the effect of each of the pipeline's steps here :
TifPipelineWidget.view_pipeline(tp,acq=acq)

# Run pipeline on the whole experience !

In [9]:
tp.process(exp)